In [11]:
from creating_arrays import CHART_TIME_ARRAY, MONTH_ARRAY, SYMBOL_ARRAY
from constants import BINANCE_URL, download_dir, output_dir, dtypes
from pathlib import Path
import os
import re
import shutil
from selenium import webdriver
import talib
import datetime
from selenium.webdriver.chrome.options import Options
import glob
import zipfile
import pandas as pd
from pandas.api.types import is_numeric_dtype
import inspect
import talib
import time
import numpy as np
from sklearn.ensemble import RandomForestClassifier


In [8]:
# Get the current date
now = datetime.datetime.now()

# Format the date as a string in the desired format
date_string = now.strftime("%Y-%m")

# Print the date string
print(date_string)

idx = MONTH_ARRAY.index(date_string)
MONTH_ARRAY = MONTH_ARRAY[:idx + 1]
months = MONTH_ARRAY

2023-01


In [24]:
# Set up the arrays of symbols, chart times, and months
symbols = [
    SYMBOL_ARRAY[SYMBOL_ARRAY.index('BTCUSDT')],
    #     SYMBOL_ARRAY[SYMBOL_ARRAY.index('ETHUSDT')],
    #     SYMBOL_ARRAY[SYMBOL_ARRAY.index('ETHBUSD')],
    #     SYMBOL_ARRAY[SYMBOL_ARRAY.index('BTCBUSD')]
]
# symbols = [SYMBOL_ARRAY[SYMBOL_ARRAY.index('BTCUSDT')]]

# chart_times = CHART_TIME_ARRAY
chart_times = [
    CHART_TIME_ARRAY[CHART_TIME_ARRAY.index('5m')],
    CHART_TIME_ARRAY[CHART_TIME_ARRAY.index('1m')],
    CHART_TIME_ARRAY[CHART_TIME_ARRAY.index('15m')]
]


In [20]:
# # Iterate over the symbols and chart times
# for symbol in symbols:
#     for chart_time in chart_times:
#         # Construct the file name
#         file_name = f"{symbol}-{chart_time}.csv"

#         # Construct the file path
#         file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

#         # Load the data into a dataframe
#         df = pd.read_csv(file_path)
#         df = df[33:]
# # display(df[33:])

## LONGS


In [26]:
# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Load the data into a dataframe
        df = pd.read_csv(file_path)
        df = df[33:]

        # Define the number of bins for RSI
        num_bins_rsi = 40

        # Create bins for RSI values
        rsi_bins = np.linspace(df["RSI"].min(), df["RSI"].max(), num_bins_rsi)

        # Create new column for RSI bin
        df["RSI_bin"] = np.digitize(df["RSI"], rsi_bins)

        # Get the number of wins in each RSI bin
        binned_data = df.groupby(["RSI_bin"])["if_long"].sum()

        # Get the total number of trades in each RSI bin
        binned_data_total = df.groupby(["RSI_bin"])["if_long"].count()

        # Calculate the win percentage in each RSI bin
        binned_data_win_percentage = binned_data / binned_data_total

        # Find RSI bin or bins with win percentage of 71% or higher. For now,
        bins_with_high_win_percentage = binned_data_win_percentage

        # Print the range of RSI and win percentage for each bin with win percentage of 71% or higher
        for rsi_bin, win_percentage in bins_with_high_win_percentage.items():
            try:
                rsi_range = (rsi_bins[rsi_bin-1], rsi_bins[rsi_bin])
                total_trades = df[(df["RSI"] >= rsi_range[0]) & (df["RSI"] < rsi_range[1])].shape[0]
                total_wins = df[(df["RSI"] >= rsi_range[0]) & (df["RSI"] < rsi_range[1]) & (df["if_long"] == 1)].shape[0]
                win_percentage = total_wins/total_trades
                if total_trades > 1000 and win_percentage > 0.6:
                    print(f"This is for the {chart_time} chart for {symbol}")
                    print("Range of RSI:", rsi_range)
                    print("Win percentage:", win_percentage)
                    print("Total trades:", total_trades)
                    print("Total wins:", total_wins)
                    print()
            except IndexError:
                pass


This is for the 5m chart for BTCUSDT
Range of RSI: (73.64599460390887, 76.15131780974365)
Win percentage: 0.6173744341452899
Total trades: 4639
Total wins: 2864

This is for the 5m chart for BTCUSDT
Range of RSI: (76.15131780974365, 78.65664101557843)
Win percentage: 0.6364692218350755
Total trades: 3444
Total wins: 2192

This is for the 5m chart for BTCUSDT
Range of RSI: (78.65664101557843, 81.1619642214132)
Win percentage: 0.6664019062748213
Total trades: 2518
Total wins: 1678

This is for the 5m chart for BTCUSDT
Range of RSI: (81.1619642214132, 83.66728742724797)
Win percentage: 0.7013422818791947
Total trades: 1788
Total wins: 1254

This is for the 5m chart for BTCUSDT
Range of RSI: (83.66728742724797, 86.17261063308275)
Win percentage: 0.730327868852459
Total trades: 1220
Total wins: 891

This is for the 1m chart for BTCUSDT
Range of RSI: (87.16546834081109, 89.72142394697083)
Win percentage: 0.6227725762609484
Total trades: 3311
Total wins: 2062

This is for the 1m chart for BTC

# SHORTS


In [ ]:
# Iterate over the symbols and chart times
for symbol in symbols:
    for chart_time in chart_times:
        # Construct the file name
        file_name = f"{symbol}-{chart_time}.csv"

        # Construct the file path
        file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

        # Load the data into a dataframe
        df = pd.read_csv(file_path)
        df = df[33:]

        # Define the number of bins for RSI
        num_bins_rsi = 20

        # Create bins for RSI values
        rsi_bins = np.linspace(df["RSI"].min(), df["RSI"].max(), num_bins_rsi)

        # Create new column for RSI bin
        df["RSI_bin"] = np.digitize(df["RSI"], rsi_bins)

        # Get the number of wins in each RSI bin
        binned_data = df.groupby(["RSI_bin"])["if_short"].sum()

        # Get the total number of trades in each RSI bin
        binned_data_total = df.groupby(["RSI_bin"])["if_short"].count()

        # Calculate the win percentage in each RSI bin
        binned_data_win_percentage = binned_data / binned_data_total

        # Find RSI bin or bins with win percentage of 71% or higher
        bins_with_high_win_percentage = binned_data_win_percentage[binned_data_win_percentage >= 0]

        # Print the range of RSI and win percentage for each bin with win percentage of 71% or higher
        for rsi_bin, win_percentage in bins_with_high_win_percentage.items():
            rsi_range = (rsi_bins[rsi_bin-1], rsi_bins[rsi_bin])
            total_trades = df[(df["RSI"] >= rsi_range[0]) & (df["RSI"] < rsi_range[1])].shape[0]
            total_wins = df[(df["RSI"] >= rsi_range[0]) & (df["RSI"] < rsi_range[1]) & (df["if_short"] == 1)].shape[0]
            print("Range of RSI:", rsi_range)
            print("Win percentage:", total_wins/total_trades)
            print("Total trades:", total_trades)
            print("Total wins:", total_wins)
            print()



In [ ]:
# # Iterate over the symbols and chart times
# for symbol in symbols:
#     for chart_time in chart_times:
#         # Construct the file name
#         file_name = f"{symbol}-{chart_time}.csv"

#         # Construct the file path
#         file_path = Path(output_dir) / f"{symbol}-{chart_time}/{file_name}"

#         # Load the data into a dataframe
#         df = pd.read_csv(file_path)
#         df = df[33:]

#         # Define the features and target variables
#         X = df[["RSI"]]
#         y = df["if_long"]

#         # Create a random forest classifier
#         clf = RandomForestClassifier(n_estimators=100, random_state=0)

#         # Fit the model to the data
#         clf.fit(X, y)

#         # Get the feature importances
#         importances = clf.feature_importances_

#         # Print the feature importances
#         print("Feature importances for symbol:", symbol, "chart_time",
#               chart_time, ":", importances)

#         # Define the number of bins for RSI
#         num_bins_rsi = 20

#         # Create bins for RSI values
#         rsi_bins = np.linspace(df["RSI"].min(), df["RSI"].max(), num_bins_rsi)

#         # Create new column for RSI bin
#         df["RSI_bin"] = np.digitize(df["RSI"], rsi_bins)

#         # Get the number of wins in each RSI bin
#         binned_data = df.groupby(["RSI_bin"])["if_long"].sum()

#         # Get the total number of trades in each RSI bin
#         binned_data_total = df.groupby(["RSI_bin"])["if_long"].count()

#         # Calculate the win percentage in each RSI bin
#         binned_data_win_percentage = binned_data / binned_data_total

#         # Find RSI bin or bins with win percentage of 71% or higher. For now,
#         bins_with_high_win_percentage = binned_data_win_percentage[binned_data_win_percentage >= 0]

#         # Print the range of RSI and win percentage for each bin with win percentage of 71% or higher
#         for rsi_bin, win_percentage in bins_with_high_win_percentage.items():
#             rsi_range = (rsi_bins[rsi_bin-1], rsi_bins[rsi_bin])
# #             rsi_range = (df["RSI"].min(), df["RSI"].max())
#             total_trades = df[(df["RSI"] >= rsi_range[0]) & (df["RSI"] < rsi_range[1])].shape[0]
#             total_wins = df[(df["RSI"] >= rsi_range[0]) & (df["RSI"] < rsi_range[1]) & (df["if_long"] == 1)].shape[0]
#             print("Range of RSI:", rsi_range)
#             print("Win percentage:", total_wins/total_trades)
#             print("Total trades:", total_trades)
#             print("Total wins:", total_wins)
#             print()
